In [10]:
from apify_client import ApifyClient
import json
import pandas as pd
import sqlite3
import json
from credentials import APIFY_API_KEY
import logging
from datetime import datetime

In [40]:
def fetch_data():
    client = ApifyClient(APIFY_API_KEY)
    
    hashtags = ['gorpcore', 'goretexstudio', 'outdoorism', 'gorpcorefashion', 'arcteryx', 'gorpcorestyle', 'outdoorism', 'itsbetteroutside']
    
    posts = []
    post_skip_count = 0

    # call db function to check if id in db
    existing_ids = get_existing_ids()
    
    for hashtag in hashtags:
        run_input = {
            "hashtags": [hashtag],
            "resultsLimit": 1000,
        }
        
        run = client.actor("apify/instagram-hashtag-scraper").call(run_input=run_input)
        
        for item in client.dataset(run["defaultDatasetId"]).iterate_items():
            try:
                if item['likesCount'] >= min_likes and item['id'] not in existing_ids:
                    posts.append(item)
                else:
                    post_skip_count += 1
            except Exception as e:
                # If an exception occurs skip this item
                print(f"Error in processing item: {e}")

    print(f"Skipped {post_skip_count} posts")
    print(f'Fetched {len(posts)} items.')
    return posts
    
db_path = "D:\coding\instagram\dbs\insta_hashtag_data1.db"

def get_existing_ids():
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    
    # Get the list of existing post IDs from the db
    cur.execute("SELECT id FROM insta_hashtag_data1;")
    existing_ids = {row[0] for row in cur.fetchall()}
    

    
    return existing_ids

# Set up logging
logging.basicConfig(filename='db_operations.log', level=logging.INFO)

def insert_db(data):

    if not data:
        print("No data to insert.")
        logging.info("No data to insert.")
        return

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    insert_query = """
    INSERT OR IGNORE INTO insta_hashtag_data1 (
        id, type, shortCode, caption, hashtags, mentions, url, commentsCount, 
        dimensionsHeight, dimensionsWidth, 
        displayUrl, images, alt, likesCount, timestamp, ownerId
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """
        
    rows_inserted = 0

    # Insert into db
    for item in data:
        try:
             values = (
                item['id'],
                item.get('type', 'default_type'), 
                item.get('shortCode', 'default_shortCode'),  
                item.get('caption', 'default_caption'),  
                json.dumps(item.get('hashtags', [])),  
                json.dumps(item.get('mentions', [])),  
                item.get('url', 'default_url'), 
                item.get('commentsCount', 0),  
                item.get('dimensionsHeight', 0), 
                item.get('dimensionsWidth', 0),  
                item.get('displayUrl', 'default_displayUrl'), 
                json.dumps(item.get('images', [])), 
                item.get('alt', 'default_alt'),  
                item.get('likesCount', 0), 
                item.get('timestamp', 'default_timestamp'),  
                item.get('ownerId', 'default_ownerId'),  
            )
            # Execute the SQL query inside the loop
             cur.execute(insert_query, values)
             if cur.rowcount > 0:
                 rows_inserted += 1
        except Exception as e:
            # If an exception occurs print a message and skip this item
            print(f"Error processing item: {e} - {item}")
            continue

    conn.commit()
    conn.close()

    # Log and print the number of rows inserted
    logging.info(f'{rows_inserted} rows inserted.')
    print(f'{rows_inserted} rows inserted.')

def db_summary():
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    
    # Get the total number of rows in the table
    cur.execute("SELECT COUNT(*) FROM insta_hashtag_data1;")
    total_rows = cur.fetchone()[0]
    
    conn.close()
    
    # Log and print the database summary
    logging.info(f'Database summary: {total_rows} rows in total.')
    print(f'Database summary: {total_rows} rows in total.')

In [41]:
def main():
      
    min_likes = 200 
    existing_ids = get_existing_ids() 
    new_data = fetch_data()
    get_existing_ids()
    insert_db(new_data)
    db_summary()

    print("Data pipeline executed successfully.")

if __name__ == "__main__":
    main()

Error in processing data: 'id'
No data to insert.
Database summary: 362 rows in total.
Data pipeline executed successfully.


In [2]:
conn = sqlite3.connect("D:\coding\instagram\dbs\insta_hashtag_data1.db")
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print("Tables in the database:", tables)

Tables in the database: [('insta_hashtag_data1',)]


In [8]:
%reload_ext sql
%sql sqlite:///D:\coding\instagram\dbs\insta_hashtag_data1.db

In [19]:
%sql SELECT * FROM insta_hashtag_data1;

 * sqlite:///D:\coding\instagram\dbs\insta_hashtag_data1.db
   sqlite:///insta_hashtag_data1.db
Done.


id,type,shortCode,caption,hashtags,mentions,url,commentsCount,dimensionsHeight,dimensionsWidth,displayUrl,images,alt,likesCount,timestamp,ownerId
3188322762416707387,Sidecar,Cw_MqjzI9c7,⅋ 💧Via. @pippo_krg —————————————————-𝘍𝘰𝘭𝘭𝘰𝘸 @𝘨𝘰𝘳𝘱𝘸𝘰𝘳𝘭𝘥 𝘧𝘰𝘳 𝘮𝘰𝘳𝘦 𝘤𝘰𝘯𝘵𝘦𝘯𝘵........#ootd #arcteryx #outdoor #outdoors #explore #gorpcore #goretexstudio #hypebeast #grailed #fashionmoodboard #outdoorlife #hiking #itsbetteroutside #outdoorism #getoutside #ordinaryfits #trekking #mountain #outfitoftheday #urban #gorpworld,"[""ootd"", ""arcteryx"", ""outdoor"", ""outdoors"", ""explore"", ""gorpcore"", ""goretexstudio"", ""hypebeast"", ""grailed"", ""fashionmoodboard"", ""outdoorlife"", ""hiking"", ""itsbetteroutside"", ""outdoorism"", ""getoutside"", ""ordinaryfits"", ""trekking"", ""mountain"", ""outfitoftheday"", ""urban"", ""gorpworld""]","[""pippo_krg"", ""\ud835\ude28\ud835\ude30\ud835\ude33\ud835\ude31\ud835\ude38\ud835\ude30\ud835\ude33\ud835\ude2d\ud835\ude25""]",https://www.instagram.com/p/Cw_MqjzI9c7/,2,1349,1080,https://instagram.fcai19-5.fna.fbcdn.net/v/t51.2885-15/375942548_270274575887123_7430616790757438220_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fcai19-5.fna.fbcdn.net&_nc_cat=111&_nc_ohc=kestE07k-e0AX-7v4IB&edm=AOUPxh0BAAAA&ccb=7-5&oh=00_AfAt9zkjw5crW3V7ceukf7TyGYxzi1TDke1C2DDnd5mlvg&oe=65057580&_nc_sid=9dc660,[],None,1183,2023-09-09T22:15:13.000Z,56846806518
3185714020269225926,Sidecar,Cw17gXWIPfG,⅋ 🎒Via. @hildurkarlsson—————————————————-𝘍𝘰𝘭𝘭𝘰𝘸 @𝙜𝙤𝙧𝙥𝙬𝙤𝙧𝙡𝙙 𝘧𝘰𝘳 𝘮𝘰𝘳𝘦 𝘤𝘰𝘯𝘵𝘦𝘯𝘵........#ootd #arcteryx #outdoor #outdoors #explore #gorpcore #goretexstudio #hypebeast #grailed #fashionmoodboard #outdoorlife #hiking #itsbetteroutside #gorpworld,"[""ootd"", ""arcteryx"", ""outdoor"", ""outdoors"", ""explore"", ""gorpcore"", ""goretexstudio"", ""hypebeast"", ""grailed"", ""fashionmoodboard"", ""outdoorlife"", ""hiking"", ""itsbetteroutside"", ""gorpworld""]","[""hildurkarlsson"", ""\ud835\ude5c\ud835\ude64\ud835\ude67\ud835\ude65\ud835\ude6c\ud835\ude64\ud835\ude67\ud835\ude61\ud835\ude59""]",https://www.instagram.com/p/Cw17gXWIPfG/,0,1349,1080,https://instagram.fcai19-5.fna.fbcdn.net/v/t51.2885-15/376254158_1105225810458590_2664973761642733429_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fcai19-5.fna.fbcdn.net&_nc_cat=102&_nc_ohc=2Oc2CqWY1zEAX_dLhno&edm=AOUPxh0BAAAA&ccb=7-5&oh=00_AfDM6JpL7qWnkZz_LNU2jj8w3f5Jb3DyYCtApiHPn-gEBw&oe=65056177&_nc_sid=9dc660,[],None,513,2023-09-06T07:52:06.000Z,56846806518
3186837456669124123,Sidecar,Cw568hBrCYb,⅋ 🦿Via. @acgtribe —————————————————-𝘍𝘰𝘭𝘭𝘰𝘸 @𝙜𝙤𝙧𝙥𝙬𝙤𝙧𝙡𝙙 𝘧𝘰𝘳 𝘮𝘰𝘳𝘦 𝘤𝘰𝘯𝘵𝘦𝘯𝘵.........#ootd #arcteryx #outdoor #outdoors #explore #gorpcore #goretexstudio #hypebeast #grailed #fashionmoodboard #outdoorlife #hiking #itsbetteroutside #outdoorism #getoutside #gorpworld,"[""ootd"", ""arcteryx"", ""outdoor"", ""outdoors"", ""explore"", ""gorpcore"", ""goretexstudio"", ""hypebeast"", ""grailed"", ""fashionmoodboard"", ""outdoorlife"", ""hiking"", ""itsbetteroutside"", ""outdoorism"", ""getoutside"", ""gorpworld""]","[""acgtribe"", ""\ud835\ude5c\ud835\ude64\ud835\ude67\ud835\ude65\ud835\ude6c\ud835\ude64\ud835\ude67\ud835\ude61\ud835\ude59""]",https://www.instagram.com/p/Cw568hBrCYb/,0,750,750,https://instagram.fcai19-5.fna.fbcdn.net/v/t51.2885-15/375526757_643052390956524_4513422467246009061_n.jpg?stp=dst-jpg_e35&_nc_ht=instagram.fcai19-5.fna.fbcdn.net&_nc_cat=100&_nc_ohc=RCud9tU-u-wAX_ikcn3&edm=AOUPxh0BAAAA&ccb=7-5&oh=00_AfBghF6FPGkmolbJOvUrQld0Z2Hae1X4nxhHQGMyJI147g&oe=65055E95&_nc_sid=9dc660,[],None,978,2023-09-07T21:04:10.000Z,56846806518
3185359075826039765,Sidecar,Cw0qzPkrXvV,⅋ 🎹Via. @iamtolgah —————————————————-𝘍𝘰𝘭𝘭𝘰𝘸 @𝙜𝙤𝙧𝙥𝙬𝙤𝙧𝙡𝙙 𝘧𝘰𝘳 𝘮𝘰𝘳𝘦 𝘤𝘰𝘯𝘵𝘦𝘯𝘵.........#ootd #arcteryx #outdoor #outdoors #explore #gorpcore #goretexstudio #hypebeast #grailed #fashionmoodboard #outdoorlife #hiking #itsbetteroutside #outdoorism #getoutside #gorpworld,"[""ootd"", ""arcteryx"", ""outdoor"", ""outdoors"", ""explore"", ""gorpcore"", ""goretexstudio"", ""hypebeast"", ""grailed"", ""fashionmoodboard"", ""outdoorlife"", ""hiking"